In [ ]:
import pandas as pd
import requests as req

import json
import csv
from random import choice
import time
import traceback

In [ ]:
listar_todos_pokemons = "https://pokeapi.co/api/v2/pokemon?limit=100000&offset=0"
dados_pokemon = "https://pokeapi.co/api/v2/pokemon/{}"
image_pokemon = "https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/other/official-artwork/{}.png"
especie_pokemon = "https://pokeapi.co/api/v2/pokemon-species/{}"
tipo_pokemon = "https://pokeapi.co/api/v2/type/{}"
naturezas = ["hardy", "docile", "serious", "bashful", "quirky"]

geracoes_permitidas = ["generation-i", "generation-ii", "generation-iii", "generation-iv"]

In [ ]:
response = req.get(listar_todos_pokemons)

lista_pokemons = response.json()['results']

print(len(lista_pokemons))

In [ ]:
movimentos_permitidos = []
with open("dados/move_list.csv",'r') as f:
    csvD = csv.DictReader(f)
    for item in csvD:
      movimentos_permitidos.append(item['identifier'])

print(len(movimentos_permitidos))

In [ ]:
habilidades_permitidas = []
with open("dados/abilities.csv",'r') as f:
    csvD = csv.DictReader(f)
    for item in csvD:
      habilidades_permitidas.append(item['ability_name'])

In [ ]:
def processa_pokemon(pokemon):
    record = None
    tipos = {}

    try:
        response = req.get(pokemon["url"])
        data = json.loads(response.text)

        response_especie = req.get(especie_pokemon.format(data["species"]["name"]))
        especie = json.loads(response_especie.text)

        if especie["generation"]["name"] not in geracoes_permitidas:
            return None

        if data["types"][0]["type"]["name"] not in tipos:
            response_tipo1 = req.get(tipo_pokemon.format(data["types"][0]["type"]["name"]))
            tipo1 = json.loads(response_tipo1.text)
            tipos[tipo1['name']] = {}
            tipos[tipo1['name']]['recebe_dano_duplo'] = [x["name"] for x in tipo1['damage_relations']['double_damage_from']]
            tipos[tipo1['name']]['recebe_dano_metade'] = [x["name"] for x in tipo1['damage_relations']['half_damage_from']]
            tipos[tipo1['name']]['recebe_dano_zero'] = [x["name"] for x in tipo1['damage_relations']['no_damage_from']]
            tipos[tipo1['name']]['causa_dano_duplo'] = [x["name"] for x in tipo1['damage_relations']['double_damage_to']]
            tipos[tipo1['name']]['causa_dano_metade'] = [x["name"] for x in tipo1['damage_relations']['half_damage_to']]
            tipos[tipo1['name']]['causa_dano_zero'] = [x["name"] for x in tipo1['damage_relations']['no_damage_to']]
            tipos[tipo1['name']]["movimentos"] = [x["name"] for x in tipo1['moves'] if x["name"] in movimentos_permitidos]

        if len(data["types"]) > 1 and data["types"][1]["type"]["name"] not in tipos:
            response_tipo2 = req.get(tipo_pokemon.format(data["types"][1]["type"]["name"]))
            tipo2 = json.loads(response_tipo2.text)
            tipos[tipo2["name"]] = {}
            tipos[tipo2["name"]]["recebe_dano_duplo"] = [x["name"] for x in tipo2["damage_relations"]["double_damage_from"]]
            tipos[tipo2["name"]]["recebe_dano_metade"] = [x["name"] for x in tipo2["damage_relations"]["half_damage_from"]]
            tipos[tipo2["name"]]["recebe_dano_zero"] = [x["name"] for x in tipo2["damage_relations"]["no_damage_from"]]
            tipos[tipo2["name"]]["causa_dano_duplo"] = [x["name"] for x in tipo2["damage_relations"]["double_damage_to"]]
            tipos[tipo2["name"]]["causa_dano_metade"] = [x["name"] for x in tipo2["damage_relations"]["half_damage_to"]]
            tipos[tipo2["name"]]["causa_dano_zero"] = [x["name"] for x in tipo2["damage_relations"]["no_damage_to"]]
            tipos[tipo2["name"]]["movimentos"] = [x["name"] for x in tipo2["moves"] if x["name"] in movimentos_permitidos]

        record = dict(
        id=data["id"],
        nome=data["name"],
        nivel=1,
        sexo=choice(["male", "female", "genderless"]),
        tipos = [x['type']['name'] for x in data['types']],
        movimentos=[],
        habilidade=None,
        natureza=choice(naturezas),
        cur_hp=data["stats"][0]["base_stat"],
        stats=[
            data["stats"][0]["base_stat"],
            data["stats"][1]["base_stat"],
            data["stats"][2]["base_stat"],
            data["stats"][4]["base_stat"],
            data["stats"][3]["base_stat"],
            data["stats"][5]["base_stat"],
        ],
        imagem=image_pokemon.format(data["id"]),
        legendario=especie["is_legendary"],
        mitico=especie["is_mythical"],
        geracao=especie["generation"]["name"],
        danos = {
            "recebe_dano_duplo": [],
            "recebe_dano_metade": [],
            "recebe_dano_zero": [],
            "causa_dano_duplo": [],
            "causa_dano_metade": [],
            "causa_dano_zero": []
        }
    )

        possible_moves = []
        for ptype in record["tipos"]:
            possible_moves += tipos[ptype]["movimentos"]
            record["danos"]["recebe_dano_duplo"] += tipos[ptype]["recebe_dano_duplo"]
            record["danos"]["recebe_dano_metade"] += tipos[ptype]["recebe_dano_metade"]
            record["danos"]["recebe_dano_zero"] += tipos[ptype]["recebe_dano_zero"]
            record["danos"]["causa_dano_duplo"] += tipos[ptype]["causa_dano_duplo"]
            record["danos"]["causa_dano_metade"] += tipos[ptype]["causa_dano_metade"]
            record["danos"]["causa_dano_zero"] += tipos[ptype]["causa_dano_zero"]

        record["movimentos"] = [choice(possible_moves) for x in range(2)]
        habilidades = [
            x["ability"]["name"]
            for x in data["abilities"]
            if x["ability"]["name"] in habilidades_permitidas
        ]
        record["habilidade"] = choice(habilidades) if len(habilidades) > 0 else None

        record["danos"]["recebe_dano_duplo"] = list(set(record["danos"]["recebe_dano_duplo"]))
        record["danos"]["recebe_dano_metade"] = list(set(record["danos"]["recebe_dano_metade"]))
        record["danos"]["recebe_dano_zero"] = list(set(record["danos"]["recebe_dano_zero"]))
        record["danos"]["causa_dano_duplo"] = list(set(record["danos"]["causa_dano_duplo"]))
        record["danos"]["causa_dano_metade"] = list(set(record["danos"]["causa_dano_metade"]))
        record["danos"]["causa_dano_zero"] = list(set(record["danos"]["causa_dano_zero"]))
    except Exception as e:
        print(f"Erro ao processar {pokemon['name']}")
        print(traceback.format_exc())

    return record

In [ ]:
results = []
errors = []

max = 25
idx = 0
wait = 2
for pokemon in lista_pokemons:
  poke = processa_pokemon(pokemon)
  if poke:
    results.append(poke)
  else:
    errors.append(pokemon)  

  idx += 1
  if idx == max:
      print(f"Processados {max} pokemons")
      time.sleep(wait)
      idx = 0

In [ ]:
with open("dados/pokemons.json", "w") as f:
    json.dump(results, f)
    
with open("dados/errors.json", "w") as f:
    json.dump(errors, f)

In [ ]:
with open("dados/pokemons.json", "r") as f:
    pokemons = json.load(f)
    pokemon_nao_lendarios = []
    for pokemon in pokemons:
        if not pokemon["legendario"]:
            pokemon_nao_lendarios.append(pokemon)

with open("dados/pokemons.json", "w") as f:
    json.dump(pokemon_nao_lendarios, f)